In [56]:
import os
from dotenv import load_dotenv
import weaviate

load_dotenv()

weaviate_apikey = os.getenv("WEAVIATE_APIKEY")
client = weaviate.Client(
    "http://localhost:8080",
    auth_client_secret=weaviate.auth.AuthApiKey(weaviate_apikey),
)
client.query.aggregate("passage").with_meta_count().do()


{'data': {'Aggregate': {'Passage': [{'meta': {'count': 64}}]}}}

In [ ]:
client.query.get("Passage", ["text_content"]).do()